# PDF Acquisition

In [ ]:
#@title Depandances
%%capture
!pip3 install -U scidownl
# Langchain and groq
!pip install langchain
!pip install langchain_groq

#installing marker
!pip install marker-pdf

In [4]:
from scidownl import scihub_download
import pandas as pd
import os
from google.colab import files

def download_paper():
    paper = input("Please enter the DOI of the paper here: ")
    paper_type = "doi"
    out = "./paper.pdf"
    # Uncomment and configure the proxies if needed
    # proxies = {
    #     'http': 'socks5://127.0.0.1:7890'
    # }
    scihub_download(paper, paper_type=paper_type, out=out)#, proxies=proxies)

def upload_paper():
    uploaded = files.upload()
    if not list(uploaded.keys())[0].endswith('.pdf'):
        raise ValueError('Uploaded file is not a PDF file.')
    os.rename(list(uploaded.keys())[0], 'paper.pdf')
    print("Paper uploaded successfully as 'paper.pdf'")

def main():
    choice = input("Choose an option (1 for DOI download, 2 for PDF upload): ")
    if choice == '1':
        download_paper()
    elif choice == '2':
        upload_paper()
    else:
        print("Invalid choice. Please enter 1 or 2.")

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

# Markdown Extraction with Marker

In [ ]:
#@title extracting markdown
!marker_single /content/paper.pdf /content/output/ --batch_multiplier 2 --max_pages 20 --langs English

# Extracting the Methodology Section
**Objective:** Isolate the methodology section from the segmented markdown for further analysis.
- **Action Items:**
  - From the parsed sections, specifically extract the text under the "Methodology" heading.
  - Handle variations in heading titles like "Methods" or "Materials and Methods."


In [ ]:
import re

def extract_outline(file_path):
    outline = []
    with open(file_path, 'r') as file:
        for line in file:
            header_match = re.match(r'^(#+)\s+(.*)', line)
            if header_match:
                level = len(header_match.group(1))
                title = header_match.group(2)
                outline.append((header_match.group(1), title))
    return outline

def print_outline(outline):
    for hashes, title in outline:
        print(hashes + ' ' + title)

file_path = 'manuscript.md'
outline = extract_outline(file_path)
print_outline(outline)


In [3]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.prompts import ChatPromptTemplate

from prompts import system_prompt_1, system_prompt_2, get_markdown

import re
import os 
from IPython.display import Markdown

from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq



# setup google gemini api keys
from dotenv import load_dotenv
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

gem1 = ChatGoogleGenerativeAI(model="gemini-1.5-pro", google_api_key=GOOGLE_API_KEY, temperature=0.4, convert_system_message_to_human=True)
flash = ChatGoogleGenerativeAI(model="gemini-1.5-flash", google_api_key=GOOGLE_API_KEY, temperature=0.3, convert_system_message_to_human=True)


GROQ_API_KEY = os.getenv("GROQ_API_KEY")

groq_mx = ChatGroq(
    temperature=0,
    # model_name="mixtral-8x7b-32768",
    # model_name="llama3-70b-8192",
    # model_name="llama-3.1-70b-versatile",
    # model_name="mixtral-8x7b-32768",
    # model_name="llama3-8b-8192",
    model_name="llama-3.1-8b-instant",
    # model_name="gemma2-9b-it",
    
    groq_api_key=GROQ_API_KEY 
)

In [ ]:




system = """
###instructions###
You will receive an outline of a scientific paper in the format shown in the expected input. Extract and present only the highest level heading corresponding to the methods section, with no other output.

**Expected Input:**
#abstract
#introduction
#method
##method 1
#results
#conclusion

**Expected Output:**
#method

"""

text = f"""{outline}"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", text)])

chain = prompt | chat
response = chain.invoke({})
print(response.content)

# Chapter Two: Materials and Methods


In [ ]:
#@title Extracting the method section from the Markdown file
import re

def extract_section(file_path, section_title):
    section_content = []
    inside_section = False
    section_level = None

    with open(file_path, 'r') as file:
        for line in file:
            header_match = re.match(r'^(#+)\s+(.*)', line)
            if header_match:
                level = len(header_match.group(1))
                title = header_match.group(2)

                if inside_section:
                    if level <= section_level:
                        break
                if header_match.group(1) + ' ' + title == section_title:
                    inside_section = True
                    section_level = level

            if inside_section:
                section_content.append(line)

    return ''.join(section_content)


file_path = 'manuscript.md'
section_title = response.content
method = extract_section(file_path, section_title)

## Comperhencive Sammary

In [1]:
from dotenv import load_dotenv
import os
load_dotenv()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")


GROQ_API_KEY = os.getenv("GROQ_API_KEY")


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


In [1]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq




In [25]:
from prompts import system_prompt_1, system_prompt_2
from input import method_1, method_2

In [20]:
#@title creation of nodes and edges

chat = ChatGroq(
    temperature=0,
    model_name="mixtral-8x7b-32768",
    
    max_tokens=1024,
    groq_api_key=GROQ_API_KEY  # Optional if not set as an environment variable
)

system = system_prompt_1

text = f"""{method_2}"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", text)])

chain = prompt | chat
nande = chain.invoke({})


In [21]:
from prompts import system_prompt_1, system_prompt_2
from input import method_1, method_2

#@title writing the code for the graphviz

chat = ChatGroq(
    temperature=0,
    model_name="gemma2-9b-it",
    max_tokens=1024,
    groq_api_key=GROQ_API_KEY  # Optional if not set as an environment variable
)

system = system_prompt_2

text = f"""{nande.content}"""
prompt = ChatPromptTemplate.from_messages([("system", system), ("human", text)])

chain = prompt | chat
llm_response = chain.invoke({})


In [22]:
#@title extract the code form the response and run it
import re

def extract_code(llm_response):
    # Define the pattern to match the Python code block
    pattern = re.compile(r'```python(.*?)```', re.DOTALL)

    # Search for the code block
    match = pattern.search(llm_response)

    if match:
        # Extract the code
        code = match.group(1).strip()
        return code
    else:
        return "No Python code found in the response."

# Example LLM response
code = extract_code(llm_response.content)
exec(code)

In [26]:
print(system_prompt_2)


### instructions ###
Your job is to write Python code that:
1. Reads the provided list of nodes and edges.
2. Utilizes Graphviz to create a directed graph.
3. Adds nodes to the graph with appropriate labels.
4. Adds edges between the nodes with descriptions of their relationships.
5. Sets aesthetic properties such as layout direction, size, shape, or other visual attributes.
7. Very important to output the results to high quality png image name "graph.png". 




# Comperhencive sammary

In [2]:
## Manuscript to pandas
import util
import importlib
importlib.reload(util)
markdown_path = 'Examples/systematic_review/systematic_review.md'

df = util.create_dataframe_from_sections(markdown_path)

In [11]:
llm = ChatGroq(
    temperature=0,
    # model_name="mixtral-8x7b-32768",
    # model_name="llama3-70b-8192",
    # model_name="llama-3.1-70b-versatile",
    # model_name="mixtral-8x7b-32768",
    model_name="llama3-8b-8192",
    # model_name="llama-3.1-8b-instant",
    # model_name="gemma2-9b-it",
    
    groq_api_key=GROQ_API_KEY 
)

In [12]:
def generate_summary(content):
    system = """
###instructions###
You are a sammarizer you job is to sammarize any input into a structured markdown sammary that is comberhencive.

## don't include a conclusion section

"""

    prompt = ChatPromptTemplate.from_messages([("system", system), ("human", content)])
    
    chain = prompt | llm
    response = chain.invoke({})
    return response.content

In [13]:



# Add the summary column using a for loop
for index, row in df.iterrows():
    df.at[index, 'summary'] = generate_summary(row['content'])


# Display the DataFrame with the new summary column
print(df)

BadRequestError: Error code: 400 - {'error': {'message': 'Please reduce the length of the messages or completion.', 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [18]:
import util
import importlib
importlib.reload(util)

def count_citations_and_generate_summary(df):
    # Function to check if the title contains reference-related words and count DOI occurrences
    reference_keywords = ['References', 'Citation', 'Citations', 'Bibliography', 'References', 'Reference']
    
    # Check if the title contains any reference-related words
    def is_reference_title(title):
        return any(keyword.lower() in title.lower() for keyword in reference_keywords)
    
    # Count DOI occurrences in the content
    def count_doi(content):
        return content.lower().count('doi')
    
    # Create new columns for citation counts and summaries
    df['summary'] = ""
    
    for index, row in df.iterrows():
        print(row['title'])
        if is_reference_title(row['title']):
            citations = count_doi(row['content'])
            df.at[index, 'summary'] = "Number of citaiton: {citations}"
        else:
            df.at[index, 'summary'] = generate_summary(row['content'])
    
    return df

# Apply the function to the dataframe
df_with_citations_and_summaries = count_citations_and_generate_summary(df)


# Molecular Docking As A Potential Approach In Repurposing Drugs Against Covid‑19: A Systematic Review And Novel Pharmacophore Models
## Abstract
## Introduction
## Methods Workfow Search Strategy
## Data Selection And Extraction
## Pharmacophore Analysis
## Data Description
## Results And Discussion Main Protease
## Antivirals
## Tetracyclines
## Pharmacophore Analysis Of The Main Protease
## Rna‑Dependent Rna Polymerase
## Drug Afecting Rna‑Dependent Rna Polymerase
## Pharmacophore Analysis Of The Rna‑Dependent Rna Polymerase
## Spike Protein
## Drug Afecting Spike Protein
## Drug Afecting The Ace2/Spike Complex
## Pharmacophore Analysis Of The Spike Protein
## Conclusion
## Declarations
## References


ModuleNotFoundError: No module named 'ace_tools'

In [16]:
# Concatenate the text in the 'sammary' column, filtering out None values
concatenated_text = ' '.join(filter(None, df['summary'].tolist()))

Markdown(concatenated_text)

**Molecular Docking as a Potential Approach in Repurposing Drugs Against COVID-19: A Systematic Review and Novel Pharmacophore Models**
=====================================================

### Abstract

This systematic review explores the potential of molecular docking as an approach to repurpose drugs against COVID-19. The study aims to identify novel pharmacophore models that can aid in the discovery of new therapeutic agents.

### Introduction

The COVID-19 pandemic has highlighted the need for innovative approaches to drug discovery and repurposing. Molecular docking is a promising technique that can be used to identify potential therapeutic agents by predicting their binding affinity to target proteins.

### Methods

A systematic review was conducted to identify relevant studies on molecular docking and COVID-19. The search strategy included major databases and keywords related to molecular docking, COVID-19, and drug repurposing.

### Results

The review identified several studies that have used molecular docking to identify potential therapeutic agents against COVID-19. The results showed that molecular docking can be a valuable tool in the discovery of new therapeutic agents.

### Novel Pharmacophore Models

The study also developed novel pharmacophore models that can be used to identify potential therapeutic agents against COVID-19. The models were developed using a combination of molecular docking and machine learning algorithms.

### Discussion

The results of this study demonstrate the potential of molecular docking as an approach to repurpose drugs against COVID-19. The novel pharmacophore models developed in this study can be used to identify potential therapeutic agents and aid in the discovery of new therapeutic agents.

### References

[Insert references]

### Figure

![0_image_0.png](0_image_0.png)

Note: The figure is not included in this summary as it is an image. **Summary of Recent Literature on Repurposing FDA-Approved Drugs for COVID-19 Treatment**
======================================================

### Purpose
The purpose of this review is to summarize the recent literature on FDA-approved drugs that have been repurposed as potential candidates against COVID-19. The study also includes a quality pharmacophore analysis of frequently studied targets, including the main protease, RNA-dependent RNA polymerase, and spike protein.

### Recent Findings
The COVID-19 pandemic has highlighted the need for effective therapeutic agents. Despite ongoing research, no absolute therapeutic agent has been developed. Drug repurposing strategies offer a viable approach to develop novel therapeutic interventions.

### Drug Repurposing
Drug repurposing involves exploring previously approved drugs with known safety and pharmacokinetic profiles for new effects, reducing costs, time, and predicting potential side effects and drug interactions. The COVID-19 viral machinery shares similarities with other viruses, making antiviral agents widely repurposed in the search for curative candidates.

### Pharmacophore Analysis
The main protease pharmacophoric study revealed multiple features, providing a starting point for future research.

**Keywords**
- COVID-19
- Drug repurposing
- Pharmacophore analysis
- Main protease
- Spike proteins
- RNA-dependent RNA polymerase **Summary**
===============

**Introduction**
---------------

In late 2019, a novel beta coronavirus was identified as the cause of severe pneumonia cases in Wuhan, China. The World Health Organization (WHO) labeled the disease as COVID-19 and the virus as SARS-CoV-2.

**Global Pandemic**
-----------------

By March 2020, the WHO announced that COVID-19 had become a global pandemic, affecting the entire world. Due to its airborne infectivity, social distancing measures were imposed by international authorities.

**Drug Repurposing**
-----------------

In response to the sudden outbreak, global research efforts focused on investigating potential therapeutic approaches. Drug repurposing strategies were implemented as an immediate drug discovery pipeline. Clinically useful drugs, such as aspirin and sildenafil, were successfully repurposed for cardiovascular diseases and erectile dysfunction, respectively.

**In Silico Methods**
-------------------

Molecular docking and pharmacophore modeling are popular in silico methods used in drug discovery research. Molecular docking illustrates interactions between ligands and biological targets, while pharmacophore modeling identifies molecular features that enable biologically active ligands to exert a pharmacological effect.

**Research Summary**
-------------------

This article reviews and summarizes research on FDA-approved drugs repurposed against SARS-CoV-2 virus. **Methods Workflow Search Strategy**
===============================

### Background
During the COVID-19 pandemic, the scientific community published a vast number of studies on repurposing drugs for COVID-19. These studies are scattered across the internet, making it challenging to find a comprehensive database that includes all of them, aside from Google Scholar.

### Search Strategy
To achieve an efficient search strategy, the following combinations of keywords were applied during the research in the Google Scholar database:

* `(COVID19 OR SARS-CoV-2)` to capture studies related to COVID-19
* `(Drug repurposing OR Drug repositioning OR Drug re-profiling OR Drug rediscovery)` to focus on studies about repurposing drugs
* `(Docking AND Molecular dynamic)` to narrow down the search to studies that involve docking and molecular dynamics

### Search Parameters
The search was conducted up to December 30, 2020, to ensure that all relevant studies were included. **Data Selection And Extraction**
=============================

### Methodology

* Titles and abstracts were reviewed for eligibility using specific inclusion and exclusion criteria (Table 1)
* Selected papers were further reviewed in a second stage, while articles that met the exclusion criteria were eliminated

### Search Results
-----------------

* Initial search yielded 405 articles
* After applying inclusion and exclusion criteria, 92 research articles were selected
* The selection process is presented in a flow diagram (Fig. 1)

### Classification
-----------------

* The 92 selected articles were classified according to their target:
	+ 73 articles focused on the main protease
	+ 11 articles focused on the spike protein
	+ 8 articles focused on the replicase complex **Pharmacophore Analysis**
======================

### Introduction

To perform pharmacophore analysis, it was essential to ensure that all analyzed drugs bind to the same site on the target. Therefore, drugs were classified within the same target according to the binding site.

### Binding Site Definition

Various approaches are used in literature to define the binding site or docking site. In this study, one binding site was obtained for each target, except for the main protease, which yielded two binding sites.

### Pharmacophore Generation

The Phase tool from Schrödinger was used to generate the pharmacophore hypothesis. The quality of the pharmacophore was a major concern, and the number of features and Hyposcore were used as measures of quality.

### Quality Criteria

The following criteria were implemented to ensure the quality of the pharmacophore:

* Maximum of 7 features
* Minimum of 6 features
* Minimum coverage of 50% of the compounds at the question **Summary of In Silico Work on COVID-19 Drug Discovery**
======================================================

### Background

In silico work on COVID-19 drug discovery began before the release of the PDB structure of SARS-CoV-2 targets. Homology modeling was used to generate protein structures, focusing on the spike protein, RNA-dependent RNA polymerase (RdRp), main protease, helicase, and papain-like protease. This summary reviews the most studied targets: main protease, spike protein, and RdRp.

### Methods

Papers followed a similar protocol, using either docking alone or docking and molecular dynamics simulation. The most used software for molecular docking was Autodock vina, followed by Glide and Autodock 4. Autodock vina was the most used due to its ease of use, open-source feature, and accuracy.

### Molecular Dynamics Simulation

Molecular dynamics (MD) simulation takes the ligand-protein complex as a dynamic module, searching for the most stable conformation. Only 48 out of 89 studies confirmed docking results with MD.

### Inclusion and Exclusion Criteria

The following criteria were used to select articles:

| Parameter | Inclusion Criteria | Exclusion Criteria |
| --- | --- | --- |
| Type of publication | Original article | Preprint, review article, letters, conference abstracts |
| Kind of study | Studies that evaluated antiviral activities against COVID-19 | Studies that evaluated antiviral activities against other viruses |
| Method of study | Docking, molecular dynamic | Homology modeling |
| Type of molecule | Drugs | Phytochemicals, non-drug molecules |
| Language | English | Language other than English |

### Flow Diagram

The flow diagram (Fig. 1) illustrates the screening process of articles included in the study.

### Software Used

Autodock vina was the most used software for molecular docking, followed by Glide and Autodock 4. **Results and Discussion: Main Protease**
================================================

### Main Protease Target

The main protease of SARS-CoV-2 has been identified as a well-positioned drug target due to its unique cleavage activity, which is not mimicked by human proteases [13••].

### Sequence Conservation

The main protease of SARS-CoV-2 shows a high sequence identity of 96% with other SARS-CoV viruses [14], encouraging researchers to study inhibitors and repurpose existing drugs against the SARS-CoV-2 main protease.

### Crystal Structure

The crystal structure of the main protease was first released by Yang's group (PDB ID: 6LU7) [15].

### Drug Classification

Drugs were classified according to the Anatomical Therapeutic Chemical (ATC) classification system (Table 2).

### Results

* Anti-infective agents were the most frequent class, with antivirals and tetracycline being the predominant subclasses.
* Antineoplastic and immunomodulating agents were the next most frequent classes (Table 2). **Antivirals**
================

### Introduction

The study investigates the potential of 29 antiviral drugs that bind with the viral main protease to also inhibit SARS-COV-2 protease. Of these, 16 drugs are already protease inhibitors, including indinavir, ritonavir, lopinavir, and others.

### Methodology

The study used docking software to analyze the binding of these drugs with the SARS-COV-2 protease. Indinavir, ritonavir, and lopinavir were tested multiple times with different software, providing evidence for their potential as COVID-19 drugs.

### Results

* 16 drugs are already protease inhibitors, including indinavir, ritonavir, lopinavir, and others.
* Indinavir, ritonavir, and lopinavir were tested multiple times with different software, providing evidence for their potential as COVID-19 drugs.
* Asunaprevir, ciluprevir, and pleconaril work on "Genome polyprotein" and could be considered for further analysis.
* The remaining 10 drugs have diverse targets and failed pharmacophore analysis, and were excluded from further analysis.

### Discussion

The study provides evidence for the potential of certain antiviral drugs to inhibit SARS-COV-2 protease, and suggests that further analysis of asunaprevir, ciluprevir, and pleconaril may be warranted. **Tetracyclines**
================

### Overview

Five tetracyclines have been reported to exhibit inhibitory activity against the main protease: oxytetracycline, lymecycline, tetracycline, minocycline, and doxycycline.

### Structure and Mechanism of Action

Tetracyclines are broad-spectrum antibiotics with a general structure consisting of four fused rings. They work by inhibiting protein synthesis, specifically by preventing the binding of aminoacyl-tRNA to the mRNA translation complex. Here is a summarized version of the text in a structured markdown format:

**Pharmacophore Analysis of the Main Protease**
=============================================

### Introduction

The main protease has been studied extensively, with most studies focusing on the 6LU7 structure. This study aims to analyze the pharmacophore of the main protease and identify potential inhibitors.

### Results

* The pharmacophore of the main protease was analyzed using a dataset of 10,000 compounds.
* The results showed that tetracycline and N3 inhibitor were the most similar pair of hypotheses, with 4 matched features.
* The pharmacophores of tetracycline and N3 inhibitor were found to align with at least 3 matches, except for N3 inhibitor and tetracycline, which aligned with 4 matched features.
* The overlapping features of N3 and tetracycline were identified as 2 hydrogen bond donors, 1 hydrogen bond acceptor, and 1 aromatic ring (Fig. 3a).

### Discussion

* The results suggest that tetracycline and N3 inhibitor are potential inhibitors of the main protease.
* The pharmacophore analysis provides a framework for identifying potential inhibitors of the main protease.
* Further studies are needed to validate the results and identify additional inhibitors.

### References

* [17, 18•, 24•, 25]
* [34, 35]
* [19, 46]
* [31]
* [16]
* [20]
* [21]
* [22]
* [23•]
* [26]
* [27]
* [28]
* [29]
* [30•]
* [32]
* [33]
* [38•]
* [39]
* [40]
* [41]
* [42]
* [43]
* [44]
* [45]
* [46]
* [47]
* [48]
* [49]
* [50]
* [51]
* [52]
* [53]
* [54]
* [55]
* [56]
* [57]
* [58]
* [59]
* [60]
* [61]
* [62]
* [63]
* [64]
* [65]
* [66]
* [67]
* [68•]
* [69]
* [70]
* [71]
* [72]
* [73]
* [74]
* [75]
* [76]
* [77]
* [78]
* [79]
* [80]
* [81]
* [82•]
* [83]
* [84]
* [85]
* [86]
* [87]
* [88]

Note: The references are listed at the end of the summary. **RNA-Dependent RNA Polymerase (RdRp)**
===============================

### Overview

RNA-dependent RNA polymerase (RdRp) is a replicase that synthesizes a complementary RNA strand using RNA as a template. The SARS-CoV-2 RdRp is composed of 932 amino acids, divided into the N-terminal and polymerase domains.

### Structure

The RdRp protein is structured into three subunits: the finger, palm, and thumb. The active site is located in the middle of the substrate domain, where RNA synthesis occurs.

### Function

The RdRp enzyme synthesizes RNA by accessing an RNA template from the template input channel and nucleoside triphosphate (NTP) from the NTP input channel.

### Target for Drug Development

RdRp has been proposed as an important target for developing drugs against coronavirus. **Drug Afecting Rna-Dependent Rna Polymerase**
===============================

### Introduction

A review of literature revealed several papers discussing Nsp12, a key enzyme in the SARS-CoV-2 replication process. The study focused on antiviral agents that target the polymerase, as well as antineoplastic agents with potential inhibitory effects.

### Antiviral Agents

Five antiviral agents were identified as targeting the polymerase: remdesivir, tipranavir, tegobuvir, simeprevir, and flibuvir. These agents have diverse mechanisms of action and viral targets.

### Remdesivir

Remdesivir is a nucleoside analog that has been extensively studied in vitro and in silico. It is incorporated into the growing viral RNA, terminating RNA replication. Remdesivir has high affinity for the active site of RdRp, competing with natural substrates and getting incorporated into the RNA chain. It has been investigated in multiple COVID-19 clinical trials and has received FDA Emergency Use Authorization for COVID-19 treatment.

### Other Reported Drugs

While other reported drugs have no similarity in their original clinical use, in silico docking data suggests a definitive interaction with the Nsp12 active site, making them promising inhibitors.

### Pharmacophore Analysis

A pharmacophore analysis was performed based on drugs and the common binding site, despite the limited available data. **Pharmacophore Analysis of RNA-Dependent RNA Polymerase**
======================================================

### Introduction

A pharmacophore analysis was conducted on eight FDA-approved drugs of miscellaneous pharmacological classes to identify potential targets for the RNA-dependent RNA polymerase enzyme.

### Results

The analysis revealed a six-feature pharmacophore, with the following features:
* Hydrogen-bond donor: 1
* Hydrogen-bond acceptor: 3
* Aromatic ring: 2

The most frequent target observed was the PDB ID 6M71.

### Comparison with Drug-Based Analysis

The results of the pharmacophore analysis were compared with those of a drug-based analysis. The two analyses disagreed on the presence of hydrogen-bond donors and acceptors, but agreed on the presence of aromatic rings.

### Pharmacophore Features

The pharmacophore analysis identified seven features, including:
* 3 hydrogen-bond donors
* 1 hydrogen-bond acceptor
* 2 aromatic rings
* 1 negative ionic feature (unique to the binding-site based hypothesis study)

### Figure 3

![8_image_0.png](8_image_0.png)

**Figure 3: Pharmacophore Analysis for Targeting SARS-CoV-2 Enzymes**

a. Main protease
b. RNA-dependent RNA polymerase **Spike Protein**
================

### Overview

The spike protein is a class fusion glycoprotein located on the surface of SARS-CoV-2. It plays a crucial role in viral infection by recognizing the host angiotensin-converting enzyme-2 receptor (ACE2).

### Structure

The spike protein consists of two subunits, S1 and S2. The S1 subunit contains a receptor-binding domain (RBD) that binds and recognizes the ACE2 receptor, while the S2 subunit mediates viral cell membrane fusion.

### Function

The spike protein uses the ACE-2 receptor for entry to the host cell. It is coated with polysaccharide molecules to evade the surveillance of the host immune system during entry. Targeting the spike protein can potentially decrease the chance of infection by preventing viral invasion to host cells.

### Repurposed Drugs

Research has been conducted to identify the role of repurposed drugs on spike proteins and ACE2. Table 4 lists drugs that target RNA-dependent RNA polymerase, while Table 5 classifies studies that interrupt the COVID-19 mechanism of entry into three classes:

* Drugs that bind to the spike protein and cannot recognize ACE2
* Drugs that bind to ACE2 and compete with the spike protein on the ACE2 binding site
* Drugs that bind to the complex of spike and ACE2 and prevent the transition to the active state

### Classification

According to the ATC classification system, most of the drugs affecting the spike protein, ACE2, and the complex fall in the anti-infective category.

### References

[92-97, 98-100, 102••, 106, 88] **Drug Affecting Spike Protein**
=============================

### Overview

Thirty-four drugs have been reported to target the spike protein, with 6 of them being antivirals and 3 of them being nucleoside analogs that originally target DNA synthesis in viruses.

### List of Drugs

| PDB ID | Drug | References |
| --- | --- | --- |
| 6M17 | Ivermectin | [16] |
| 6LZG | Hydroxychloroquine, Chloroquine | [101] |
| 6VW1 | Troxerutin | [102••] |
| 6VSB | Dolutegravir | [102••, 103] |
| 6M17 | Cangrelor | [102••, 103, 104••] |
| 6M17 | Kanamycin | [104••] |
| 6VSB | Nelfnavir | [104••, 105] |
| 2AJF | Lopinavir | [106] |
| 6LZG | Hydroxychloroquine | [101] |
| 6VSB | Daclatasvir | [42, 102••] |
| 6M0J | Polymyxin B | [88] |
| 6M17 | Pralatrexate | [107] |
| 6VXX | Imipenem | [107, 108••] |
| 6VSB | Streptomycin | [53] |

### Notes

* Ivermectin is reported twice in the spike docking analysis, but its frequency of appearance does not indicate a potential for being a lead compound.
* Some in silico trials targeted ivermectin from the beginning, rather than as the end result of an exhaustive virtual screening. **Drug Afecting The ACE2/Spike Complex**
=====================================

### Overview

Several studies have investigated the binding of drugs to the ACE2/spike complex, aiming to disrupt the interaction of ACE2 with RBD after spike binding has already occurred. Eight drugs have been reported to target this complex, including:

* Naltrexone
* Ivermectin
* Lopinavir
* Chloroquine
* Polymyxin B
* Colistin
* Daptomycin
* Oritavancin

### Characteristics of Targeted Drugs

* Molecular weight above 500, with most above 1000
* Pharmacophore analysis failed due to complexities and variability

### Studies Targeting ACE2 Receptor Only

Two studies have investigated compounds that prevent the recognition of spike to ACE2. Ten drugs were reported to bind well to ACE2, including:

* Lividomycin
* Burixafor
* Quisinostat
* Fuprofylline
* Pemetrexed
* Spirofylline
* Edotecarin
* Diniprofylline
* Hydroxychloroquine
* Chloroquine

Note: These compounds may bind to multiple targets, including ACE2. **Pharmacophore Analysis of the Spike Protein**
=============================================

### Background

Pharmacophore analysis was conducted on the spike protein, but unlike the main protease, there was no consistency in the use of a common PDB structure from the paper that studied the spike protein.

### Methodology

A single PDB structure with the highest resolution was chosen for binding site pharmacophore analysis, as the sequences of the PDB structures were similar and thus likely to have similar structures.

### Results

The pharmacophore analysis failed due to the diversity of the drugs used. **Summary**
================

**Introduction**
---------------

The current research on drug repurposing using molecular docking and in silico drug discovery approaches in the fight against COVID-19 is reviewed.

**Main Findings**
----------------

* Antiviral drugs were the most studied drugs.
* The main protease was identified as a potential target due to its central role in viral replication and availability.
* Pharmacophore investigation showed promising results with tetracyclines and antivirals, which were originally protease inhibitors.

**Recommendations**
-------------------

* The models should be used in further research, including virtual screening, to aid in finding effective therapeutic agents against COVID-19.

**Author Contributions**
----------------------

All authors listed have significantly contributed to the development and writing of this article. **Declarations**
================

### Conflict of Interest

The authors declare no competing interests.

### Human and Animal Rights and Informed Consent

This article does not contain any studies with human or animal subjects performed by any of the authors.

In [19]:
df_with_citations_and_summaries

,title,content,citation_count,summary
0,# Molecular Docking As A Potential Approach In...,Molecular Docking As A Potential Approach In R...,0,**Molecular Docking as a Potential Approach in...
1,## Abstract,Abstract\n\nPurpose of Review This article pro...,0,**Summary of Recent Literature on Repurposing ...
2,## Introduction,"Introduction\n\nTowards the end of 2019, a nov...",0,**Summary**\n===============\n\n**Introduction...
3,## Methods Workfow Search Strategy,Methods Workfow Search Strategy\n\nDuring the ...,0,**Methods Workflow Search Strategy**\n========...
4,## Data Selection And Extraction,Data Selection And Extraction\n\nTitles and ab...,0,**Data Selection And Extraction**\n===========...
5,## Pharmacophore Analysis,Pharmacophore Analysis\n\nTo perform the pharm...,0,**Pharmacophore Analysis**\n==================...
6,## Data Description,Data Description\n\nIn silico work on COVID-19...,0,**Summary of In Silico Work on COVID-19 Drug D...
7,## Results And Discussion Main Protease,Results And Discussion Main Protease\n\nThe fa...,0,**Results and Discussion: Main Protease**\n===...
8,## Antivirals,Antivirals\n\nOf the 29 antiviral drugs that w...,0,**Antivirals**\n================\n\n### Introd...
9,## Tetracyclines,Tetracyclines\n\nAnother class of compounds th...,0,**Tetracyclines**\n================\n\n### Ove...
